## NHL: Logistic Classification

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import turicreate as tc
import os.path as path
from sklearn import preprocessing
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)
warnings.simplefilter(action = 'ignore', category = UserWarning)

In [2]:
# Settings to allow all columns in dataframe to display
desired_width = 320
pd.set_option('display.width', desired_width)
np.set_printoptions(linewidth=desired_width)
pd.set_option('display.max_columns', 111)

In [3]:
# Import dataset
allTeamsSeasons = pd.read_csv('~/desktop/data_science/NHL/Datasets/all_teams.csv')
allTeamsSeasons.head()

,team,season,name,gameId,playerTeam,opposingTeam,home_or_away,gameDate,position,situation,xGoalsPercentage,corsiPercentage,fenwickPercentage,iceTime,xOnGoalFor,xGoalsFor,xReboundsFor,xFreezeFor,xPlayStoppedFor,xPlayContinuedInZoneFor,xPlayContinuedOutsideZoneFor,flurryAdjustedxGoalsFor,scoreVenueAdjustedxGoalsFor,flurryScoreVenueAdjustedxGoalsFor,shotsOnGoalFor,missedShotsFor,blockedShotAttemptsFor,shotAttemptsFor,goalsFor,reboundsFor,reboundGoalsFor,freezeFor,playStoppedFor,playContinuedInZoneFor,playContinuedOutsideZoneFor,savedShotsOnGoalFor,savedUnblockedShotAttemptsFor,penaltiesFor,penalityMinutesFor,faceOffsWonFor,hitsFor,takeawaysFor,giveawaysFor,lowDangerShotsFor,mediumDangerShotsFor,highDangerShotsFor,lowDangerxGoalsFor,mediumDangerxGoalsFor,highDangerxGoalsFor,lowDangerGoalsFor,mediumDangerGoalsFor,highDangerGoalsFor,scoreAdjustedShotsAttemptsFor,unblockedShotAttemptsFor,scoreAdjustedUnblockedShotAttemptsFor,dZoneGiveawaysFor,xGoalsFromxReboundsOfShotsFor,xGoalsFromActualReboundsOfShotsFor,reboundxGoalsFor,totalShotCreditFor,scoreAdjustedTotalShotCreditFor,scoreFlurryAdjustedTotalShotCreditFor,xOnGoalAgainst,xGoalsAgainst,xReboundsAgainst,xFreezeAgainst,xPlayStoppedAgainst,xPlayContinuedInZoneAgainst,xPlayContinuedOutsideZoneAgainst,flurryAdjustedxGoalsAgainst,scoreVenueAdjustedxGoalsAgainst,flurryScoreVenueAdjustedxGoalsAgainst,shotsOnGoalAgainst,missedShotsAgainst,blockedShotAttemptsAgainst,shotAttemptsAgainst,goalsAgainst,reboundsAgainst,reboundGoalsAgainst,freezeAgainst,playStoppedAgainst,playContinuedInZoneAgainst,playContinuedOutsideZoneAgainst,savedShotsOnGoalAgainst,savedUnblockedShotAttemptsAgainst,penaltiesAgainst,penalityMinutesAgainst,faceOffsWonAgainst,hitsAgainst,takeawaysAgainst,giveawaysAgainst,lowDangerShotsAgainst,mediumDangerShotsAgainst,highDangerShotsAgainst,lowDangerxGoalsAgainst,mediumDangerxGoalsAgainst,highDangerxGoalsAgainst,lowDangerGoalsAgainst,mediumDangerGoalsAgainst,highDangerGoalsAgainst,scoreAdjustedShotsAttemptsAgainst,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,playoffGame
0,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,other,0.0000,0.0000,0.0000,42.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.689,0.019,0.069,0.164,0.020,0.521,0.207,0.019,0.019,0.019,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.019,0.000,0.000,0.0,0.0,0.0,1.00,1.0,1.000,0.0,0.017,0.000,0.000,0.037,0.037,0.037,0
1,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,all,0.4596,0.6408,0.6310,3600.0,37.733,2.287,2.842,8.155,1.134,22.530,16.054,2.249,2.396,2.357,41.0,12.0,13.0,66.0,2.0,1.0,0.0,14.0,2.0,18.0,16.0,39.0,51.0,4.0,8.0,30.0,16.0,19.0,8.0,44.0,8.0,1.0,1.040,0.811,0.435,1.0,0.0,1.0,68.292,53.0,54.697,4.0,0.641,0.014,0.029,2.899,3.030,2.980,22.880,2.689,1.780,4.436,0.639,12.626,8.830,2.524,2.607,2.450,21.0,10.0,6.0,37.0,1.0,1.0,0.0,5.0,1.0,8.0,15.0,20.0,30.0,7.0,14.0,32.0,17.0,11.0,7.0,16.0,12.0,3.0,0.599,1.278,0.811,1.0,0.0,0.0,36.14,31.0,30.369,5.0,0.396,0.168,0.168,2.917,2.833,2.714,0
2,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,5on5,0.4857,0.6429,0.6364,2283.0,24.802,1.693,1.724,5.513,0.739,14.650,10.681,1.656,1.803,1.764,28.0,7.0,10.0,45.0,1.0,0.0,0.0,10.0,1.0,14.0,9.0,27.0,34.0,3.0,6.0,19.0,13.0,13.0,7.0,27.0,8.0,0.0,0.882,0.811,0.000,1.0,0.0,0.0,47.292,35.0,36.697,4.0,0.355,0.000,0.000,2.048,2.180,2.130,14.729,1.793,1.117,3.001,0.406,7.737,5.946,1.633,1.711,1.559,11.0,9.0,5.0,25.0,1.0,1.0,0.0,4.0,1.0,4.0,9.0,10.0,19.0,7.0,14.0,18.0,13.0,9.0,5.0,12.0,6.0,2.0,0.498,0.689,0.606,1.0,0.0,0.0,24.14,20.0,19.369

## Data Cleaning

In [4]:
# Confirm number of opposingTeams is correct
print('Number of Teams:', allTeamsSeasons.opposingTeam.nunique())
print('List of acronyms:', sorted(allTeamsSeasons['opposingTeam'].unique()))

Number of Teams: 37
List of acronyms: ['ANA', 'ARI', 'ATL', 'BOS', 'BUF', 'CAR', 'CBJ', 'CGY', 'CHI', 'COL', 'DAL', 'DET', 'EDM', 'FLA', 'L.A', 'LAK', 'MIN', 'MTL', 'N.J', 'NJD', 'NSH', 'NYI', 'NYR', 'OTT', 'PHI', 'PIT', 'S.J', 'SEA', 'SJS', 'STL', 'T.B', 'TBL', 'TOR', 'VAN', 'VGK', 'WPG', 'WSH']


In [5]:
# Replacing duplicate acronyms
allTeamsSeasons['opposingTeam'].replace({'S.J': 'SJS'}, inplace = True)
allTeamsSeasons['opposingTeam'].replace({'N.J': 'NJD'}, inplace = True)
allTeamsSeasons['opposingTeam'].replace({'T.B': 'TBL'}, inplace = True)
allTeamsSeasons['opposingTeam'].replace({'L.A': 'LAK'}, inplace = True)
allTeamsSeasons['opposingTeam'].replace({'ATL': 'WPG'}, inplace = True)

allTeamsSeasons['team'].replace({'S.J': 'SJS'}, inplace = True)
allTeamsSeasons['team'].replace({'N.J': 'NJD'}, inplace = True)
allTeamsSeasons['team'].replace({'T.B': 'TBL'}, inplace = True)
allTeamsSeasons['team'].replace({'L.A': 'LAK'}, inplace = True)
allTeamsSeasons['team'].replace({'ATL': 'WPG'}, inplace = True)

allTeamsSeasons['name'].replace({'S.J': 'SJS'}, inplace = True)
allTeamsSeasons['name'].replace({'N.J': 'NJD'}, inplace = True)
allTeamsSeasons['name'].replace({'T.B': 'TBL'}, inplace = True)
allTeamsSeasons['name'].replace({'L.A': 'LAK'}, inplace = True)
allTeamsSeasons['name'].replace({'ATL': 'WPG'}, inplace = True)

allTeamsSeasons['playerTeam'].replace({'S.J': 'SJS'}, inplace = True)
allTeamsSeasons['playerTeam'].replace({'N.J': 'NJD'}, inplace = True)
allTeamsSeasons['playerTeam'].replace({'T.B': 'TBL'}, inplace = True)
allTeamsSeasons['playerTeam'].replace({'L.A': 'LAK'}, inplace = True)
allTeamsSeasons['playerTeam'].replace({'ATL': 'WPG'}, inplace = True)

print('Number of Teams:', allTeamsSeasons.opposingTeam.nunique())
print('List of acronyms:', sorted(allTeamsSeasons['opposingTeam'].unique()))

Number of Teams: 32
List of acronyms: ['ANA', 'ARI', 'BOS', 'BUF', 'CAR', 'CBJ', 'CGY', 'CHI', 'COL', 'DAL', 'DET', 'EDM', 'FLA', 'LAK', 'MIN', 'MTL', 'NJD', 'NSH', 'NYI', 'NYR', 'OTT', 'PHI', 'PIT', 'SEA', 'SJS', 'STL', 'TBL', 'TOR', 'VAN', 'VGK', 'WPG', 'WSH']


## Feature Engineering

In [6]:
# Adding columns
shootout_game = np.where((allTeamsSeasons['situation'] == 'all') & (allTeamsSeasons['goalsFor'] == allTeamsSeasons['goalsAgainst']), 1, 0)
allTeamsSeasons.insert(loc = 6, column = 'Shootout Game', value = shootout_game)

ot_game = np.where(allTeamsSeasons['iceTime'] > 3600.0, 1, 0)
allTeamsSeasons.insert(loc = 7, column = 'OT Game', value = ot_game)

win = np.where(allTeamsSeasons['goalsFor'] > allTeamsSeasons['goalsAgainst'], 1, 0)
allTeamsSeasons.insert(loc = 8, column = 'Win', value = win)

loss = np.where((allTeamsSeasons['OT Game'] == 0) & (allTeamsSeasons['goalsFor'] < allTeamsSeasons['goalsAgainst']), 1, 0)
allTeamsSeasons.insert(loc = 9, column = 'Loss', value = loss)

# ot_loss = np.where((allTeamsSeasons['OT Game'] == 1) & (allTeamsSeasons['goalsFor'] < allTeamsSeasons['goalsAgainst']), 1, 0)
# allTeamsSeasons.insert(loc = 10, column = 'OT Loss', value = ot_loss)

allTeamsSeasons.head()

,team,season,name,gameId,playerTeam,opposingTeam,Shootout Game,OT Game,Win,Loss,home_or_away,gameDate,position,situation,xGoalsPercentage,corsiPercentage,fenwickPercentage,iceTime,xOnGoalFor,xGoalsFor,xReboundsFor,xFreezeFor,xPlayStoppedFor,xPlayContinuedInZoneFor,xPlayContinuedOutsideZoneFor,flurryAdjustedxGoalsFor,scoreVenueAdjustedxGoalsFor,flurryScoreVenueAdjustedxGoalsFor,shotsOnGoalFor,missedShotsFor,blockedShotAttemptsFor,shotAttemptsFor,goalsFor,reboundsFor,reboundGoalsFor,freezeFor,playStoppedFor,playContinuedInZoneFor,playContinuedOutsideZoneFor,savedShotsOnGoalFor,savedUnblockedShotAttemptsFor,penaltiesFor,penalityMinutesFor,faceOffsWonFor,hitsFor,takeawaysFor,giveawaysFor,lowDangerShotsFor,mediumDangerShotsFor,highDangerShotsFor,lowDangerxGoalsFor,mediumDangerxGoalsFor,highDangerxGoalsFor,lowDangerGoalsFor,mediumDangerGoalsFor,...,xGoalsFromxReboundsOfShotsFor,xGoalsFromActualReboundsOfShotsFor,reboundxGoalsFor,totalShotCreditFor,scoreAdjustedTotalShotCreditFor,scoreFlurryAdjustedTotalShotCreditFor,xOnGoalAgainst,xGoalsAgainst,xReboundsAgainst,xFreezeAgainst,xPlayStoppedAgainst,xPlayContinuedInZoneAgainst,xPlayContinuedOutsideZoneAgainst,flurryAdjustedxGoalsAgainst,scoreVenueAdjustedxGoalsAgainst,flurryScoreVenueAdjustedxGoalsAgainst,shotsOnGoalAgainst,missedShotsAgainst,blockedShotAttemptsAgainst,shotAttemptsAgainst,goalsAgainst,reboundsAgainst,reboundGoalsAgainst,freezeAgainst,playStoppedAgainst,playContinuedInZoneAgainst,playContinuedOutsideZoneAgainst,savedShotsOnGoalAgainst,savedUnblockedShotAttemptsAgainst,penaltiesAgainst,penalityMinutesAgainst,faceOffsWonAgainst,hitsAgainst,takeawaysAgainst,giveawaysAgainst,lowDangerShotsAgainst,mediumDangerShotsAgainst,highDangerShotsAgainst,lowDangerxGoalsAgainst,mediumDangerxGoalsAgainst,highDangerxGoalsAgainst,lowDangerGoalsAgainst,mediumDangerGoalsAgainst,highDangerGoalsAgainst,scoreAdjustedShotsAttemptsAgainst,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,playoffGame
0,NYR,2008,NYR,2008020001,NYR,TBL,0,0,0,0,AWAY,20081004,Team Level,other,0.0000,0.0000,0.0000,42.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.689,0.019,0.069,0.164,0.020,0.521,0.207,0.019,0.019,0.019,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.019,0.000,0.000,0.0,0.0,0.0,1.00,1.0,1.000,0.0,0.017,0.000,0.000,0.037,0.037,0.037,0
1,NYR,2008,NYR,2008020001,NYR,TBL,0,0,1,0,AWAY,20081004,Team Level,all,0.4596,0.6408,0.6310,3600.0,37.733,2.287,2.842,8.155,1.134,22.530,16.054,2.249,2.396,2.357,41.0,12.0,13.0,66.0,2.0,1.0,0.0,14.0,2.0,18.0,16.0,39.0,51.0,4.0,8.0,30.0,16.0,19.0,8.0,44.0,8.0,1.0,1.040,0.811,0.435,1.0,0.0,...,0.641,0.014,0.029,2.899,3.030,2.980,22.880,2.689,1.780,4.436,0.639,12.626,8.830,2.524,2.607,2.450,21.0,10.0,6.0,37.0,1.0,1.0,0.0,5.0,1.0,8.0,15.0,20.0,30.0,7.0,14.0,32.0,17.0,11.0,7.0,16.0,12.0,3.0,0.599,1.278,0.811,1.0,0.0,0.0,36.14,31.0,30.369,5.0,0.396,0.168,0.168,2.917,2.833,2.714,0
2,NYR,2008,NYR,2008020001,NYR,TBL,0,0,0,0,AWAY,20081004,Team Level,5on5,0.4857,0.6429,0.6364,2283.0,24.802,1.693,1.724,5.513,0.739,14.650,10.681,1.656,1.803,1.764,28.0,7.0,10.0,45.0,1.0,0.0,0.0,10.0,1.0,14.0,9.0,27.0,34.0,3.0,6.0,19.0,13.0,13.0,7.0,27.0,8.0,0.0,0.882,0.811,0.000,1.0,0.0,...,0.355,0.000,0.000,2.048,2.180,2.130,14.729,1.793,1.117,3.001,0.406,7.737,5.946,1.633,1.711,1.559,11.0,9.0,5.0,25.0,1.0,1.0,0.0,4.0,1.0,4.0,9.0,10.0,19.0,7.0,14.0,18.0,13.0,9.0,5.0,12.0,6.0,2.0,0.498,0.689,0.606,1.0,0.0,0.0,24.14,20.0,19.369,3.0,0.237,0.168,0.168,1.862,1.777,1.665,0
3,NYR,2008,NYR,2008020001,NYR,TBL,0,0,0,0,AWAY,20081004,Team Level,4on5,0.0482,0.0909,0.1000,4

In [7]:
# Changing home_or_away column to be numerical
le = preprocessing.LabelEncoder()
allTeamsSeasons['home_or_away'] = le.fit_transform(allTeamsSeasons['home_or_away'])
print(le.classes_) # It numbers them in the order they are encountered

['AWAY' 'HOME']


In [8]:
# Adding date columns

allTeamsSeasons['gameDate'] = pd.to_datetime(allTeamsSeasons['gameDate'],format='%Y%m%d')
allTeamsSeasons['year'] = pd.DatetimeIndex(allTeamsSeasons['gameDate']).year
allTeamsSeasons['month'] = pd.DatetimeIndex(allTeamsSeasons['gameDate']).month
allTeamsSeasons['day'] = pd.DatetimeIndex(allTeamsSeasons['gameDate']).day

In [9]:
# Slicing the data
#homeTeam = 'SJS'
allTeamsSeasons = allTeamsSeasons[allTeamsSeasons['situation'] == 'all']
allTeamsSeasons = allTeamsSeasons[allTeamsSeasons['season'] >= 2018]
#allTeamsSeasons = allTeamsSeasons[allTeamsSeasons['team'] == homeTeam]
allTeamsSeasons = allTeamsSeasons[allTeamsSeasons['playoffGame'] == 0]
allTeamsSeasons = allTeamsSeasons[allTeamsSeasons['Shootout Game'] == 0]
allTeamsSeasons.head()

,team,season,name,gameId,playerTeam,opposingTeam,Shootout Game,OT Game,Win,Loss,home_or_away,gameDate,position,situation,xGoalsPercentage,corsiPercentage,fenwickPercentage,iceTime,xOnGoalFor,xGoalsFor,xReboundsFor,xFreezeFor,xPlayStoppedFor,xPlayContinuedInZoneFor,xPlayContinuedOutsideZoneFor,flurryAdjustedxGoalsFor,scoreVenueAdjustedxGoalsFor,flurryScoreVenueAdjustedxGoalsFor,shotsOnGoalFor,missedShotsFor,blockedShotAttemptsFor,shotAttemptsFor,goalsFor,reboundsFor,reboundGoalsFor,freezeFor,playStoppedFor,playContinuedInZoneFor,playContinuedOutsideZoneFor,savedShotsOnGoalFor,savedUnblockedShotAttemptsFor,penaltiesFor,penalityMinutesFor,faceOffsWonFor,hitsFor,takeawaysFor,giveawaysFor,lowDangerShotsFor,mediumDangerShotsFor,highDangerShotsFor,lowDangerxGoalsFor,mediumDangerxGoalsFor,highDangerxGoalsFor,lowDangerGoalsFor,mediumDangerGoalsFor,...,totalShotCreditFor,scoreAdjustedTotalShotCreditFor,scoreFlurryAdjustedTotalShotCreditFor,xOnGoalAgainst,xGoalsAgainst,xReboundsAgainst,xFreezeAgainst,xPlayStoppedAgainst,xPlayContinuedInZoneAgainst,xPlayContinuedOutsideZoneAgainst,flurryAdjustedxGoalsAgainst,scoreVenueAdjustedxGoalsAgainst,flurryScoreVenueAdjustedxGoalsAgainst,shotsOnGoalAgainst,missedShotsAgainst,blockedShotAttemptsAgainst,shotAttemptsAgainst,goalsAgainst,reboundsAgainst,reboundGoalsAgainst,freezeAgainst,playStoppedAgainst,playContinuedInZoneAgainst,playContinuedOutsideZoneAgainst,savedShotsOnGoalAgainst,savedUnblockedShotAttemptsAgainst,penaltiesAgainst,penalityMinutesAgainst,faceOffsWonAgainst,hitsAgainst,takeawaysAgainst,giveawaysAgainst,lowDangerShotsAgainst,mediumDangerShotsAgainst,highDangerShotsAgainst,lowDangerxGoalsAgainst,mediumDangerxGoalsAgainst,highDangerxGoalsAgainst,lowDangerGoalsAgainst,mediumDangerGoalsAgainst,highDangerGoalsAgainst,scoreAdjustedShotsAttemptsAgainst,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,playoffGame,year,month,day
3931,NYR,2018,NYR,2018020006,NYR,NSH,0,0,0,1,1,2018-10-04,Team Level,all,0.6160,0.4874,0.5341,3600.0,34.472,2.984,2.224,7.267,1.042,18.851,14.633,2.948,2.852,2.818,36.0,11.0,11.0,58.0,2.0,2.0,1.0,5.0,1.0,22.0,15.0,34.0,45.0,2.0,4.0,29.0,24.0,10.0,10.0,33.0,11.0,3.0,0.785,1.412,0.787,0.0,0.0,...,2.949,2.819,2.790,29.403,1.860,1.845,7.090,0.898,16.864,12.443,1.849,1.953,1.941,33.0,8.0,20.0,61.0,3.0,0.0,0.0,8.0,2.0,12.0,16.0,30.0,38.0,3.0,6.0,32.0,21.0,16.0,9.0,33.0,7.0,1.0,0.893,0.736,0.231,2.0,0.0,1.0,64.070,41.0,42.797,4.0,0.384,0.000,0.000,2.244,2.357,2.342,0,2018,10,4
3936,NYR,2018,NYR,2018020017,NYR,BUF,0,0,0,1,0,2018-10-06,Team Level,all,0.5210,0.5397,0.5500,3600.0,38.416,3.506,2.534,8.314,1.252,23.311,16.083,3.082,3.505,3.080,44.0,11.0,13.0,68.0,1.0,3.0,1.0,12.0,1.0,21.0,17.0,43.0,54.0,7.0,17.0,37.0,18.0,2.0,4.0,42.0,9.0,4.0,1.089,0.911,1.506,0.0,0.0,...,2.881,2.879,2.828,31.020,3.224,2.194,6.727,0.995,18.767,13.093,3.201,3.222,3.199,29.0,16.0,13.0,58.0,3.0,0.0,0.0,6.0,1.0,16.0,19.0,26.0,42.0,5.0,13.0,34.0,18.0,5.0,6.0,33.0,9.0,3.0,0.842,1.043,1.339,0.0,2.0,1.0,59.726,45.0,46.105,3.0,0.483,0.000,0.000,3.707,3.706,3.677,0,2018,10,6
3941,NYR,2018,NYR,2018020029,NYR,CAR,0,0,0,1,0,2018-10-07,Team Level,all,0.4291,0.4080,0.3646,3600.0,25.495,2.940,2.036,5.373,0.783,12.523,11.345,2.877,3.079,3.014,24.0,11.0,16.0,51.0,5.0,2.0,1.0,4.0,0.0,5.0,19.0,19.0,30.0,5.0,10.0,27.0,27.0,8.0,11.0,26.0,7.0,2.0,0.800,0.977,1.163,0.0,4.0,...,2.242,2.389,2.358,42.802,3.911,2.944,10.152,1.470,24.936,17.588,3.715,3.789,3.596,40.0,21.0,13.0,74.0,8.0,6.0,2.0,7.0,0.0,14.0,26.0,32.0,53.0,5.0,10.0,38.0,30.0,13.0,18.0,47.0,11.0,3.0,1.711,1.472,0.728,3.0,3.0,2.0,71.500,61.0,59.215,9.0,0.632,0.915,0.915,3.628,3.507,3.378,0,2018,10,7
3946,NYR,2018,NYR,2018020049,NYR,SJS,0,1,1,0,1,2018-10-11,Team Level,all,0.4592,0.3817,0.3776,3637.0,25.430,2.150,1.641,5.978,0.839

## Predictive Modeling - LC

In [10]:
# New DataFrame for model
df = allTeamsSeasons[['gameId', 'team', 'opposingTeam', 'goalsFor', 'goalsAgainst', 'OT Game', 'Win', 'home_or_away'
                      , 'flurryScoreVenueAdjustedxGoalsFor', 'flurryScoreVenueAdjustedxGoalsAgainst', 'year', 'month', 'day']]
print('df.shape:', df.shape)
df.head()

df.shape: (8262, 13)


,gameId,team,opposingTeam,goalsFor,goalsAgainst,OT Game,Win,home_or_away,flurryScoreVenueAdjustedxGoalsFor,flurryScoreVenueAdjustedxGoalsAgainst,year,month,day
3931,2018020006,NYR,NSH,2.0,3.0,0,0,1,2.818,1.941,2018,10,4
3936,2018020017,NYR,BUF,1.0,3.0,0,0,0,3.080,3.199,2018,10,6
3941,2018020029,NYR,CAR,5.0,8.0,0,0,0,3.014,3.596,2018,10,7
3946,2018020049,NYR,SJS,3.0,2.0,1,1,1,1.993,2.643,2018,10,11
3951,2018020057,NYR,EDM,1.0,2.0,0,0,1,1.992,2.348,2018,10,13


In [11]:
# Converting DataFrame to SFrame
wpgSF = tc.SFrame(df)
wpgSF.head()

gameId,team,opposingTeam,goalsFor,goalsAgainst,OT Game,Win,home_or_away,flurryScoreVenueAdjustedxGoalsFor ...
2018020006,NYR,NSH,2.0,3.0,0,0,1,2.818
2018020017,NYR,BUF,1.0,3.0,0,0,0,3.08
2018020029,NYR,CAR,5.0,8.0,0,0,0,3.014
2018020049,NYR,SJS,3.0,2.0,1,1,1,1.993
2018020057,NYR,EDM,1.0,2.0,0,0,1,1.992
2018020087,NYR,WSH,3.0,4.0,1,0,0,3.982
2018020115,NYR,CGY,1.0,4.0,0,0,1,3.886
2018020122,NYR,FLA,5.0,2.0,0,1,1,2.439
2018020139,NYR,CHI,1.0,4.0,0,0,0,1.911
2018020159,NYR,LAK,3.0,4.0,0,0,0,1.117


In [12]:
# Split the data into train and test data
seed = 100
train_data, test_data = wpgSF.random_split(0.775, seed = seed)

In [13]:
# Create a model
model = tc.logistic_classifier.create(train_data
                                     , target='Win'
                                     , features = ['flurryScoreVenueAdjustedxGoalsFor'
                                                  ,'flurryScoreVenueAdjustedxGoalsAgainst'
                                                  , 'home_or_away'
                                                  , 'opposingTeam'
                                                  , 'year', 'month', 'day']
                                     , l1_penalty = 0.5
                                     , l2_penalty = 1.4
                                     , max_iterations = 50
                                     , seed = seed
                                     , validation_set= 'auto'
                                     )

# Save predictions (probability estimates) to an SArray
predictions = model.classify(test_data)
# print(predictions)

# Evaluate the model and save the results into a dictionary
evaluate = model.evaluate(test_data)
# print(results)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Logistic regression:

--------------------------------------------------------

Number of examples          : 6070

Number of classes           : 2

...

| 5         | 6        | 0.000134  | 1.306814     | 0.644316          | 0.596875            |

| 10        | 11       | 0.000134  | 1.424276     | 0.684349          | 0.643750            |

| 45        | 46       | 0.000134  | 2.180860     | 0.716639          | 0.725000            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

## Predictions

In [14]:
# Making new predictions
# CREATE VARIABLES FOR 'TEAM', 'OPPOSINGTEAM', AND 'HOME_OR_AWAY'  --> PUT IN my_prediction INSTEAD
# EASIER TO UPDATE DAILY!

my_prediction = tc.SFrame({'team': ['WPG', 'BUF', 'TOR', 'MTL', 'NJD', 'NYR', 'PHI', 'PIT', 'NYI', 'STL', 'MIN', 'DAL', 'EDM', 'SEA', 'ARZ']
                 , 'opposingTeam': ['CGY', 'CHI', 'BOS', 'FLA', 'DET', 'WAS', 'OTT', 'CBJ', 'TBL', 'VGK', 'COL', 'ANA', 'VAN', 'SJS', 'NSH']
                 #, 'home_or_away': [0,0,0,0,0,0,0,0,0,0,0]})
                 , 'home_or_away': [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]})
print(my_prediction)

print('The predicted class:', model.predict(my_prediction, output_type = "class"))         
print('The probability class [1]:', model.predict(my_prediction, output_type = 'probability'))
print('The margin of the predicted class:', model.predict(my_prediction, output_type = "margin"))


+--------------+--------------+------+
| home_or_away | opposingTeam | team |
+--------------+--------------+------+
|      1       |     CGY      | WPG  |
|      1       |     CHI      | BUF  |
|      1       |     BOS      | TOR  |
|      1       |     FLA      | MTL  |
|      1       |     DET      | NJD  |
|      1       |     WAS      | NYR  |
|      1       |     OTT      | PHI  |
|      1       |     CBJ      | PIT  |
|      1       |     TBL      | NYI  |
|      1       |     VGK      | STL  |
+--------------+--------------+------+
[15 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
The predicted class: [1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]
The probability class [1]: [0.5420193521231972, 0.5531006765042441, 0.438261705430854, 0.46861988218107764, 0.5868252255563867, 0.5285236256869768, 0.6115588593909833, 0.5148211539019555, 0.38419144824103924, 0.5515854189276087, 0.460685

__Saving Model Results__

In [15]:
# print(dir(model))

# FIX OUTPUT --> USE A LOOP TO APPEND EACH PREDICTION ONE AT A TIME!!!

# Save results from current model to a dictionary
new_row = {
    'Target': getattr(model, 'target')
    , 'Team': my_prediction['team']
    , 'Opposing Team': my_prediction['opposingTeam']
    , 'Home or Away': my_prediction['home_or_away']
    , 'Prediction':  model.predict(my_prediction, output_type = "class")
    , 'Probability': model.predict(my_prediction, output_type = 'probability')
    , 'Margin': model.predict(my_prediction, output_type = "margin")
    , 'Training Accuracy': getattr(model, 'training_accuracy')
    , 'Validation Accuracy': getattr(model, 'validation_accuracy')
    , 'Testing Accuracy': evaluate['accuracy']
    , 'Log Loss': getattr(model, 'validation_log_loss')
    , 'f1-Score': getattr(model, 'validation_f1_score')
    , 'Precision': getattr(model, 'validation_precision')
    , 'Recall': getattr(model,'validation_recall')
    , 'Target': getattr(model, 'target')
    , '# of Features': getattr(model, 'num_features')
    , 'Features': getattr(model, 'features')
    , 'L1 Penalty': getattr(model, 'l1_penalty')
    , 'L2 Penalty': getattr(model, 'l2_penalty')
    , 'Random Seed': seed
    }

# Check to see if .csv exists, if not, create new DataFrame for model results
link = path.expanduser('~/Desktop/data_science/NHL/Exports/logistic_classification_results_ALL.csv')

if path.exists(link):
    results = pd.read_csv('~/Desktop/data_science/NHL/Exports/logistic_classification_results_ALL.csv')
else:
    results = pd.DataFrame()
    
# Append 'new_row' results to DataFrame
results_df = results
results_df = results_df.append(new_row, ignore_index=True)

# Save DataFrame to localhost
results_df.to_csv('~/Desktop/data_science/NHL/Exports/logistic_classification_results_ALL.csv', index = False)

results_df.tail(12)  

,Target,Team,Opposing Team,Home or Away,Prediction,Probability,Margin,Training Accuracy,Validation Accuracy,Testing Accuracy,Log Loss,f1-Score,Precision,Recall,# of Features,Features,L1 Penalty,L2 Penalty,Random Seed
66,Win,"['WPG', 'BUF', 'TOR', 'MTL', 'NJD', 'NYR', 'PH...","['CGY', 'CHI', 'BOS', 'FLA', 'DET', 'WAS', 'OT...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]","[0.5420193521231972, 0.5531006765042441, 0.438...","[0.16847477614920606, 0.21320669794639138, -0....",0.716639,0.725,0.706197,0.553502,0.716129,0.744966,0.689441,7.0,"['flurryScoreVenueAdjustedxGoalsFor', 'flurryS...",0.5,1.4,100.0
67,Win,"['WPG', 'BUF', 'TOR', 'MTL', 'NJD', 'NYR', 'PH...","['CGY', 'CHI', 'BOS', 'FLA', 'DET', 'WAS', 'OT...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]","[0.5420193521231972, 0.5531006765042441, 0.438...","[0.16847477614920606, 0.21320669794639138, -0....",0.716639,0.725,0.706197,0.553502,0.716129,0.744966,0.689441,7.0,"['flurryScoreVenueAdjustedxGoalsFor', 'flurryS...",0.5,1.4,100.0
68,Win,"['WPG', 'BUF', 'TOR', 'MTL', 'NJD', 'NYR', 'PH...","['CGY', 'CHI', 'BOS', 'FLA', 'DET', 'WAS', 'OT...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]","[0.5420193521231972, 0.5531006765042441, 0.438...","[0.16847477614920606, 0.21320669794639138, -0....",0.716639,0.725,0.706197,0.553502,0.716129,0.744966,0.689441,7.0,"['flurryScoreVenueAdjustedxGoalsFor', 'flurryS...",0.5,1.4,100.0
69,Win,"['WPG', 'BUF', 'TOR', 'MTL', 'NJD', 'NYR', 'PH...","['CGY', 'CHI', 'BOS', 'FLA', 'DET', 'WAS', 'OT...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]","[0.5420193521231972, 0.5531006765042441, 0.438...","[0.16847477614920606, 0.21320669794639138, -0....",0.716639,0.725,0.706197,0.553502,0.716129,0.744966,0.689441,7.0,"['flurryScoreVenueAdjustedxGoalsFor', 'flurryS...",0.5,1.4,100.0
70,Win,"['WPG', 'BUF', 'TOR', 'MTL', 'NJD', 'NYR', 'PH...","['CGY', 'CHI', 'BOS', 'FLA', 'DET', 'WAS', 'OT...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]","[0.5420193521231972, 0.5531006765042441, 0.438...","[0.16847477614920606, 0.21320669794639138, -0....",0.716639,0.725,0.706197,0.553502,0.716129,0.744966,0.689441,7.0,"['flurryScoreVenueAdjustedxGoalsFor', 'flurryS...",0.5,1.4,100.0
71,Win,"['WPG', 'BUF', 'TOR', 'MTL', 'NJD', 'NYR', 'PH...","['CGY', 'CHI', 'BOS', 'FLA', 'DET', 'WAS', 'OT...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]","[0.5420193521231972, 0.5531006765042441, 0.438...","[0.16847477614920606, 0.21320669794639138, -0....",0.716639,0.725,0.706197,0.553502,0.716129,0.744966,0.689441,7.0,"['flurryScoreVenueAdjustedxGoalsFor', 'flurryS...",0.5,1.4,100.0
72,Win,"['WPG', 'BUF', 'TOR', 'MTL', 'NJD', 'NYR', 'PH...","['CGY', 'CHI', 'BOS', 'FLA', 'DET', 'WAS', 'OT...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]","[0.5420193521231972, 0.5531006765042441, 0.438...","[0.16847477614920606, 0.21320669794639138, -0....",0.716639,0.725,0.706197,0.553502,0.716129,0.744966,0.689441,7.0,"['flurryScoreVenueAdjustedxGoalsFor', 'flurryS...",0.5,1.4,100.0
73,Win,"['WPG', 'BUF', 'TOR', 'MTL', 'NJD', 'NYR', 'PH...","['CGY', 'CHI', 'BOS', 'FLA', 'DET', 'WAS', 'OT...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]","[0.5420193521231972, 0.5531006765042441, 0.438...","[0.16847477614920606, 0.21320669794639138, -0....",0.716639,0.725,0.706197,0.553502,0.716129,0.744966,0.689441,7.0,"['flurryScoreVenueAdjustedxGoalsFor', 'flurryS...",0.5,1.4,100.0
74,Win,"['WPG', 'BUF', 'TOR', 'MTL', 'NJD', 'NYR', 'PH...","['CGY', 'CHI', 'BOS', 'FLA', 'DET', 'WAS', 'OT...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]","[0.5420193521231972, 0.5531006765042441, 0.438.

## K-Fold Testing

In [16]:
"""
#Importing required libraries
from sklearn.datasets import load_breast_cancer
import pandas as pd
from sklearn.model_selection import KFold 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
 
#Loading the dataset
data = pd.read_csv('~/desktop/data_science/NHL/Datasets/all_teams.csv')
df2 = data
X = df2[['flurryScoreVenueAdjustedxGoalsFor','flurryScoreVenueAdjustedxGoalsAgainst']]
labels = df2['goalsAgainst']
y = labels

#Implementing cross validation
 
k = 5
kf = KFold(n_splits=k, random_state=None)
model = LogisticRegression(solver= 'liblinear')
 
acc_score = []
 
for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    acc = accuracy_score(pred_values , y_test)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))
"""

"\n#Importing required libraries\nfrom sklearn.datasets import load_breast_cancer\nimport pandas as pd\nfrom sklearn.model_selection import KFold \nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.metrics import accuracy_score\n \n#Loading the dataset\ndata = pd.read_csv('~/desktop/data_science/NHL/Datasets/all_teams.csv')\ndf2 = data\nX = df2[['flurryScoreVenueAdjustedxGoalsFor','flurryScoreVenueAdjustedxGoalsAgainst']]\nlabels = df2['goalsAgainst']\ny = labels\n\n#Implementing cross validation\n \nk = 5\nkf = KFold(n_splits=k, random_state=None)\nmodel = LogisticRegression(solver= 'liblinear')\n \nacc_score = []\n \nfor train_index , test_index in kf.split(X):\n    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]\n    y_train , y_test = y[train_index] , y[test_index]\n     \n    model.fit(X_train,y_train)\n    pred_values = model.predict(X_test)\n     \n    acc = accuracy_score(pred_values , y_test)\n    acc_score.append(acc)\n     \navg_acc_score = sum(

## Neural Network

In [17]:
# Import libraries
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [18]:
# DataFrame for NN model
dfnn = allTeamsSeasons[['gameId', 'team', 'opposingTeam', 'goalsFor', 'goalsAgainst', 'OT Game', 'Win', 'home_or_away'
                        ,'corsiPercentage', 'fenwickPercentage', 'shotsOnGoalFor', 'missedShotsFor', 'blockedShotAttemptsFor'
                        , 'shotAttemptsFor', 'goalsFor', 'reboundsFor', 'reboundGoalsFor'
                        , 'flurryScoreVenueAdjustedxGoalsFor', 'flurryScoreVenueAdjustedxGoalsAgainst', 'year', 'month', 'day']]

In [19]:
# Changing 'team' column to be numerical
team = preprocessing.LabelEncoder()
dfnn['team'] = team.fit_transform(dfnn['team'])
print(team.classes_)

['ANA' 'ARI' 'BOS' 'BUF' 'CAR' 'CBJ' 'CGY' 'CHI' 'COL' 'DAL' 'DET' 'EDM' 'FLA' 'LAK' 'MIN' 'MTL' 'NJD' 'NSH' 'NYI' 'NYR' 'OTT' 'PHI' 'PIT' 'SEA' 'SJS' 'STL' 'TBL' 'TOR' 'VAN' 'VGK' 'WPG' 'WSH']


/var/folders/pk/zz9mgj_94m5fggpq12mdd19m0000gn/T/ipykernel_85000/901553950.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfnn['team'] = team.fit_transform(dfnn['team'])


In [20]:
# Build 'team' DataFrame for lookups

In [21]:
# Changing 'opposingTeam' column to be numerical
opposingTeam = preprocessing.LabelEncoder()
dfnn['opposingTeam'] = opposingTeam.fit_transform(dfnn['opposingTeam'])
print(opposingTeam.classes_)

['ANA' 'ARI' 'BOS' 'BUF' 'CAR' 'CBJ' 'CGY' 'CHI' 'COL' 'DAL' 'DET' 'EDM' 'FLA' 'LAK' 'MIN' 'MTL' 'NJD' 'NSH' 'NYI' 'NYR' 'OTT' 'PHI' 'PIT' 'SEA' 'SJS' 'STL' 'TBL' 'TOR' 'VAN' 'VGK' 'WPG' 'WSH']


/var/folders/pk/zz9mgj_94m5fggpq12mdd19m0000gn/T/ipykernel_85000/1109712141.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfnn['opposingTeam'] = opposingTeam.fit_transform(dfnn['opposingTeam'])


In [22]:
dfnn.head()

,gameId,team,opposingTeam,goalsFor,goalsAgainst,OT Game,Win,home_or_away,corsiPercentage,fenwickPercentage,shotsOnGoalFor,missedShotsFor,blockedShotAttemptsFor,shotAttemptsFor,goalsFor,reboundsFor,reboundGoalsFor,flurryScoreVenueAdjustedxGoalsFor,flurryScoreVenueAdjustedxGoalsAgainst,year,month,day
3931,2018020006,19,17,2.0,3.0,0,0,1,0.4874,0.5341,36.0,11.0,11.0,58.0,2.0,2.0,1.0,2.818,1.941,2018,10,4
3936,2018020017,19,3,1.0,3.0,0,0,0,0.5397,0.5500,44.0,11.0,13.0,68.0,1.0,3.0,1.0,3.080,3.199,2018,10,6
3941,2018020029,19,4,5.0,8.0,0,0,0,0.4080,0.3646,24.0,11.0,16.0,51.0,5.0,2.0,1.0,3.014,3.596,2018,10,7
3946,2018020049,19,24,3.0,2.0,1,1,1,0.3817,0.3776,27.0,10.0,13.0,50.0,3.0,0.0,0.0,1.993,2.643,2018,10,11
3951,2018020057,19,11,1.0,2.0,0,0,1,0.4811,0.4805,24.0,13.0,14.0,51.0,1.0,3.0,0.0,1.992,2.348,2018,10,13


In [23]:
# Build 'opposingTeam' DataFrame

In [24]:
# DataFrames to hold the Input Features
X_input = dfnn[['gameId', 'team', 'opposingTeam', 'goalsFor', 'goalsAgainst', 'OT Game', 'Win', 'home_or_away'
                        ,'corsiPercentage', 'fenwickPercentage', 'shotsOnGoalFor', 'missedShotsFor', 'blockedShotAttemptsFor'
                        , 'shotAttemptsFor', 'goalsFor', 'reboundsFor', 'reboundGoalsFor'
                        , 'flurryScoreVenueAdjustedxGoalsFor', 'flurryScoreVenueAdjustedxGoalsAgainst', 'year', 'month', 'day']]
X_input

,gameId,team,opposingTeam,goalsFor,goalsFor,goalsAgainst,OT Game,Win,home_or_away,corsiPercentage,fenwickPercentage,shotsOnGoalFor,missedShotsFor,blockedShotAttemptsFor,shotAttemptsFor,goalsFor,goalsFor,reboundsFor,reboundGoalsFor,flurryScoreVenueAdjustedxGoalsFor,flurryScoreVenueAdjustedxGoalsAgainst,year,month,day
3931,2018020006,19,17,2.0,2.0,3.0,0,0,1,0.4874,0.5341,36.0,11.0,11.0,58.0,2.0,2.0,2.0,1.0,2.818,1.941,2018,10,4
3936,2018020017,19,3,1.0,1.0,3.0,0,0,0,0.5397,0.5500,44.0,11.0,13.0,68.0,1.0,1.0,3.0,1.0,3.080,3.199,2018,10,6
3941,2018020029,19,4,5.0,5.0,8.0,0,0,0,0.4080,0.3646,24.0,11.0,16.0,51.0,5.0,5.0,2.0,1.0,3.014,3.596,2018,10,7
3946,2018020049,19,24,3.0,3.0,2.0,1,1,1,0.3817,0.3776,27.0,10.0,13.0,50.0,3.0,3.0,0.0,0.0,1.993,2.643,2018,10,11
3951,2018020057,19,11,1.0,1.0,2.0,0,0,1,0.4811,0.4805,24.0,13.0,14.0,51.0,1.0,1.0,3.0,0.0,1.992,2.348,2018,10,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174291,2020020852,13,8,2.0,2.0,3.0,0,0,1,0.2990,0.3000,17.0,4.0,8.0,29.0,2.0,2.0,2.0,2.0,1.314,2.869,2021,5,7
174296,2020020867,13,8,2.0,2.0,3.0,0,0,1,0.4624,0.4412,18.0,12.0,13.0,43.0,2.0,2.0,2.0,0.0,1.373,2.360,2021,5,8
174301,2020020456,13,25,1.0,1.0,2.0,1,0,1,0.5556,0.5758,27.0,11.0,17.0,55.0,1.0,1.0,1.0,0.0,1.615,2.252,2021,5,10
174306,2020020688,13,8,0.0,0.0,6.0,0,0,0,0.3761,0.3210,18.0,8.0,15.0,41.0,0.0,0.0,1.0,0.0,0.738,4.128,2021,5,12


In [25]:
# DataFrames to hold the Target Labels
Y_label = dfnn['Win'].values
Y_label

array([0, 0, 0, ..., 0, 0, 0])

In [26]:
# Using one-hot encoding to reshape labels to make them compatible for neural networks
labelencoder_Y = LabelEncoder() 
Y_label = labelencoder_Y.fit_transform(Y_label)
Y_label

array([0, 0, 0, ..., 0, 0, 0])

In [27]:
# Finding the shape for input_dim
print(f"The shape of the dataset: {X_input.shape}")
dims = X_input.shape[1]
print(f'Dimensions:{dims}')

The shape of the dataset: (8262, 24)
Dimensions:24


In [28]:
# Building a multilayer neural network using Keras
from tensorflow.keras.layers import Flatten

model = Sequential()
model.add(Dense(256, input_dim = dims, activation = 'relu')) # Why input_dim on this line only?
model.add(Dense(128, activation = 'relu'))
#model.add(Dropout(.20))
#model.add(Dense(64, input_dim = dims,  activation = 'relu'))

model.add(Dense(32, activation = 'softmax'))

2022-10-03 14:33:11.576968: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [29]:
optimizer = tf.keras.optimizers.Adam(0.1)

In [30]:
# Compile the model to provide the training parameters
model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [31]:
# Train the model
model.fit(X_input, Y_label, epochs = 2500, verbose = 1)

Epoch 1/2500
259/259 [==============================] - 1s 1ms/step - loss: 51469348.0000 - accuracy: 0.4902
Epoch 2/2500
259/259 [==============================] - 0s 995us/step - loss: 0.7016 - accuracy: 0.5046
Epoch 3/2500
259/259 [==============================] - 0s 891us/step - loss: 0.7102 - accuracy: 0.5015
Epoch 4/2500
259/259 [==============================] - 0s 865us/step - loss: 0.7081 - accuracy: 0.5061
Epoch 5/2500
259/259 [==============================] - 0s 917us/step - loss: 0.7040 - accuracy: 0.5002
Epoch 6/2500
259/259 [==============================] - 0s 1ms/step - loss: 0.7078 - accuracy: 0.4862
Epoch 7/2500
259/259 [==============================] - 0s 1ms/step - loss: 0.7025 - accuracy: 0.5005
Epoch 8/2500
259/259 [==============================] - 0s 1ms/step - loss: 0.7191 - accuracy: 0.5015
Epoch 9/2500
259/259 [==============================] - 0s 2ms/step - loss: 0.7063 - accuracy: 0.5063
Epoch 10/2500
259/259 [==============================] - 0s 2ms/ste

KeyboardInterrupt: 

In [ ]:
# Evaluate the trained model
results = model.evaluate(X_input, Y_label)
print(f"The accuracy of the model is: {results[1]}")
print(f"The loss of the model is: {round(results[0], 4)}")